# Isochrones

**Objetive:** <br>
The following notebook shows the step-by-step to creates a dataset with social infrastructure in LAC from official and public records. Public records include data from [Open Street Maps](https://www.openstreetmap.org/).<br>

Author: Laura Goyeneche, Consultant SPH, lauragoy@iadb.org <br>
Created: March 20, 2023

## 1. Basics

In [1]:
%%capture
# Libraries
import io
import os 
import re
import time
import boto3
import dotenv
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from geopandas.tools import sjoin
from OSMPythonTools.overpass import Overpass, overpassQueryBuilder
import overpy

# Modules
from utils import get_iadb

In [2]:
# Working environments
dotenv.load_dotenv("/home/ec2-user/SageMaker/.env")
sclbucket   = os.environ.get("sclbucket")
scldatalake = os.environ.get("scldatalake")

In [3]:
%%capture 
# Resources and buckets
s3        = boto3.client('s3')
s3_       = boto3.resource("s3")
s3_bucket = s3_.Bucket(sclbucket)

## 2. Inputs

In [4]:
# Country names 
data = get_iadb()

## 3. OSM amenities

In [5]:
# Initialize Overpass API
api = overpy.Overpass()

### 3.1. Financial

In [14]:
# Set parameters for the query
isoalpha3 = "COL"
amenity   = "atm"

# Define the query for fetching data for ATMs in Colombia
query = f'''
        area["ISO3166-1:alpha3"="{isoalpha3}"]->.a;
        ( node(area.a)[amenity={amenity}];
          way(area.a)[amenity={amenity}];
          rel(area.a)[amenity={amenity}];);
        out center;
        '''

# Execute the query
r = api.query(query)

# Extract coordinates and metadata for nodes, ways, and relations
    # Coordinates
coords_node     = pd.DataFrame([{'id': node.id    , 'lon': node.lon           , 'lat': node.lat}            for node     in r.nodes]).reset_index()
coords_way      = pd.DataFrame([{'id': way.id     , 'lon': way.center_lon     , 'lat': way.center_lat}      for way      in r.ways]).reset_index()
coords_relation = pd.DataFrame([{'id': relation.id, 'lon': relation.center_lon, 'lat': relation.center_lat} for relation in r.relations]).reset_index()
    
    # Metadata
metadata_node     = pd.DataFrame([node.tags     for node     in r.nodes]).reset_index()
metadata_way      = pd.DataFrame([way.tags      for way      in r.ways]).reset_index()
metadata_relation = pd.DataFrame([relation.tags for relation in r.relations]).reset_index()

# Merge coordinates and metadata for nodes, ways, and relations
output_node     = coords_node.merge(metadata_node, on = 'index').drop('index', axis = 1)
output_way      = coords_way.merge(metadata_way, on = 'index').drop('index', axis = 1)
output_relation = coords_relation.merge(metadata_relation, on = 'index').drop('index', axis = 1)

# Combine the results into a single DataFrame
output_tot_atm = pd.concat([output_node, output_way, output_relation])

### 3.2. Healthcare

In [16]:
# Set parameters for the query
isoalpha3 = "COL"
amenity   = "hospital"

# Define the query for fetching data related to hospitals in Colombia
query = f'''
        area["ISO3166-1:alpha3"="{isoalpha3}"]->.a;
        ( node(area.a)[amenity={amenity}];
          way(area.a)[amenity={amenity}];
          rel(area.a)[amenity={amenity}];);
        out center;
        '''

# Execute the query
r = api.query(query)

# Extract coordinates and metadata for nodes, ways, and relations
    # Coordinates
coords_node     = pd.DataFrame([{'id': node.id    , 'lon': node.lon           , 'lat': node.lat}            for node in r.nodes]).reset_index()
coords_way      = pd.DataFrame([{'id': way.id     , 'lon': way.center_lon     , 'lat': way.center_lat}      for way in r.ways]).reset_index()
coords_relation = pd.DataFrame([{'id': relation.id, 'lon': relation.center_lon, 'lat': relation.center_lat} for relation in r.relations]).reset_index()

    # Metadata
metadata_node     = pd.DataFrame([node.tags     for node     in r.nodes]).reset_index()
metadata_way      = pd.DataFrame([way.tags      for way      in r.ways]).reset_index()
metadata_relation = pd.DataFrame([relation.tags for relation in r.relations]).reset_index()

# Merge coordinates and metadata for nodes, ways, and relations
output_node     = coords_node.merge(metadata_node, on = 'index').drop('index', axis = 1)
output_way      = coords_way.merge(metadata_way, on = 'index').drop('index', axis = 1)
output_relation = coords_relation.merge(metadata_relation, on = 'index').drop('index', axis = 1)

# Combine the results into a single DataFrame
output_tot_hospitals = pd.concat([output_node, output_way, output_relation])

## 4. Master tables

###  4.1. Financial

In [6]:
# Files in financial bucket
path  = "Geospatial infrastructure/Financial infrastructure"
files = [file.key.split(path + "/")[1] for file in s3_bucket.objects.filter(Prefix=path).all()]

# Identify records by categories
official = [file for file in files if "official" in file]
public   = [file for file in files if "OSM"      in file]

# Create master table 
financial_inf = []

#### 4.1.1. Process official records

In [7]:
# Identify country names with official records
off_name = list(set([file.split("/")[1] for file in official]))

#### 4.1.2. Process OSM records

In [8]:
# Import data
file_  = [file for file in public if "OSM" in file]
for file in file_:
    path_ = f"{scldatalake}{path}/{file}"
    file  = pd.read_csv(path_, low_memory = False)

    # Keep IADB countries
    file = file[file.isoalpha3.isin(data.isoalpha3.unique())]

    # Keeps countries without official or healthsites.io records
    file = file[~file.isoalpha3.isin(off_name)]

    # Create variables
    file['source']    = "OSM"
    file['source_id'] = file.id

    # Keep variables of interest
    file = file[['isoalpha3','source','source_id','amenity','name','lat','lon']]

    # Add to master table
    financial_inf.append(file)

#### 4.1.3. Master table

In [9]:
# Generate master table 
financial = pd.concat(financial_inf)
financial = financial.reset_index(drop = True)

In [10]:
%%capture
# Export to Data Lake 
path = scldatalake + f"{path}/financial_facilities.csv"
financial.to_csv(path, compression = 'gzip')

In [11]:
# Temporary 
# TODO: erase once IT fix problem
financial.to_csv(f"../data/0-raw/infrastructure/financial_facilities.csv")

###  4.2. Health

In [24]:
# Files in financial bucket
path  = "Geospatial infrastructure/Healthcare Facilities"
files = [file.key.split(path + "/")[1] for file in s3_bucket.objects.filter(Prefix=path).all()]

# Identify records by categories
official = [file for file in files if "official"    in file]
public   = [file for file in files if "healthsites" in file or "OSM" in file]

# Create master table 
healthcare_inf = []

#### 4.2.1. Process official records

In [25]:
# Identify country names with official records
off_name = list(set([file.split("/")[1] for file in official]))

##### Argentina

In [26]:
# Import data
file  = [file for file in official if "ARG" in file][0]
path_ = f"{scldatalake}{path}/{file}"
file  = pd.read_csv(path_)

# Create variables
file['isoalpha3'] = "ARG"
file['source']    = "Ministry of Health"
file['source_id'] = file.establecimiento_id
file['amenity']   = "hospital"
file['name']      = file.establecimiento_nombre
file['lat']       = file.y
file['lon']       = file.x

# Keep variables of interest
file = file[file.columns[-7::]]

# Add to master table
healthcare_inf.append(file)

##### Brasil

In [27]:
%%capture
# Import data
file  = [file for file in official if "BRA" in file][0]
path_ = f"{scldatalake}{path}/{file}"
file  = pd.read_csv(path_, sep = ";", encoding = "unicode_escape")

# Create variables
file['isoalpha3'] = "BRA"
file['source']    = "Ministry of Health"
file['source_id'] = file.CO_CNES
file['amenity']   = "hospital"
file['name']      = file.NO_FANTASIA
file['lat']       = file.NU_LATITUDE
file['lon']       = file.NU_LONGITUDE

# Keep variables of interest
file = file[file.columns[-7::]]

# Add to master table
healthcare_inf.append(file)

##### Ecuador

In [28]:
# Import data
file  = [file for file in official if "ECU" in file][0]
path_ = f"{scldatalake}{path}/{file}"
file  = pd.read_csv(path_)

# Create variables
file['isoalpha3'] = "BRA"
file['source']    = "Ministry of Health"
file['source_id'] = file.unicodigo
file['amenity']   = "hospital"
file['name']      = file["nombre oficial"]
file['lat']       = file.y
file['lon']       = file.x

# Keep variables of interest
file = file[file.columns[-7::]]

# Add to master table
healthcare_inf.append(file)

##### Guyana

In [42]:
# Import data
    # Define inputs
file  = [file for file in official if "GUY" in file][0]
path_ = f"{path}/{file}"
obj   = s3.get_object(Bucket = sclbucket, Key = path_)

    # Read data
excel_data = obj['Body'].read()
excel_file = io.BytesIO(excel_data)
file       = pd.read_excel(excel_file, engine = 'openpyxl')

# Create variables
file['isoalpha3'] = "GUY"
file['source']    = "Ministry of Health"
file['source_id'] = np.nan
file['amenity']   = file["Facility Type"]
file['name']      = file.Name
file['lat']       = file[" latitude"]
file['lon']       = file[" longitude"]

# Keep variables of interest
file = file[file.columns[-7::]]

# Add to master table
healthcare_inf.append(file)

##### Jamaica

In [43]:
# Import data
file  = [file for file in official if "JAM" in file][0]
path_ = f"{scldatalake}{path}/{file}"
file  = pd.read_csv(path_)

# Create variables
file['isoalpha3'] = "JAM"
file['source']    = "Ministry of Health"
file['source_id'] = np.nan
file['amenity']   = file.Type.str.lower()
file['name']      = file[['H_Name','Parish']].apply(lambda x : '{} in {}'.format(x[0],x[1]), axis = 1)
file['lat']       = file.GeoJSON.apply(lambda x: re.findall(r"\d+\.\d+", x)[1])
file['lon']       = file.GeoJSON.apply(lambda x: re.findall(r"\d+\.\d+", x)[0])

# Keep variables of interest
file = file[file.columns[-7::]]

# Add to master table
healthcare_inf.append(file)

##### Mexico

In [45]:
# Import data
    # Define inputs
file  = [file for file in official if "MEX" in file][0]
path_ = f"{path}/{file}"
obj   = s3.get_object(Bucket = sclbucket, Key = path_)

    # Read data
excel_data = obj['Body'].read()
excel_file = io.BytesIO(excel_data)
file       = pd.read_excel(excel_file, engine = 'openpyxl')

# Create variables
file['isoalpha3'] = "MEX"
file['source']    = "Ministry of Health"
file['source_id'] = file.ID
file['amenity']   = file["NOMBRE TIPO ESTABLECIMIENTO"].str.replace("DE ","")
file['name']      = file["NOMBRE DE LA UNIDAD"]
file['lat']       = file["LATITUD"]
file['lon']       = file["LONGITUD"]

# Keep variables of interest
file = file[file.columns[-7::]]

# Add to master table
healthcare_inf.append(file)

##### Peru

In [15]:
# Import data
file  = [file for file in official if "PER" in file][0]
path_ = f"{scldatalake}{path}/{file}"
file  = pd.read_csv(path_)

# Dictionary with amenity names
n_before  = ['I-1','I-2','I-3','I-4','II-1','II-2','II-E','III-1','III-2','III-E','SD'] 
n_after   = ["Primary care"] * 4 + ["Secondary care"] * 3 + ["Tertiary care"] * 3 + [""]
d_amenity = dict(zip(n_before,n_after))

# Create variables
file['isoalpha3'] = "PER"
file['source']    = "Ministry of Health"
file['source_id'] = file.codigo_renaes
file['amenity']   = file.categoria.replace(d_amenity)
file['name']      = file[['nombre','diresa']].apply(lambda x : '{} in {}'.format(x[0].title(),x[1].title()), axis = 1)
file['lat']       = file.latitud
file['lon']       = file.longitud

# Keep variables of interest
file = file[file.columns[-7::]]

# Add to master table
healthcare_inf.append(file)

##### El Salvador

In [16]:
# Import data 
file_ = [file for file in official if "SLV" in file]
for file in file_:
    path_ = f"{scldatalake}{path}/{file}"
    file  = pd.read_csv(path_)

    # Create variables
    file['isoalpha3'] = "SLV"
    file['source']    = "Ministry of Health"
    file['source_id'] = np.nan
    file['amenity']   = file.ESPECIALIZACION.str.lower()
    file['name']      = file[['Name','MUNICIPIO','REGION']].apply(lambda x : '{}, {}, {}'.format(x[0], x[1], x[2]), axis = 1)
    file['lat']       = file.Y
    file['lon']       = file.X

    # Keep variables of interest
    file = file[file.columns[-7::]]

    # Add to master table
    healthcare_inf.append(file)

#### 4.2.2. Process healthsites records

In [17]:
# Import data
file  = [file for file in public if "healthsites" in file][0]
path_ = f"{scldatalake}{path}/{file}"
file  = pd.read_csv(path_)

# Keep rows of interest
file = file[~file.isoalpha3.isin(off_name)]
file = file[~file.isoalpha3.isna()]

# Keep variables of interest
file = file.drop(columns = "geometry")

# Identify healthsites country names
hsite_name = file.isoalpha3.unique().tolist()

# Add to master tables
healthcare_inf.append(file)

#### 4.2.3. Process OSM records

In [18]:
# Import data
file  = [file for file in public if "OSM" in file][0]
path_ = f"{scldatalake}{path}/{file}"
file  = pd.read_csv(path_, low_memory = False)

# Keep IADB countries
file = file[file.isoalpha3.isin(data.isoalpha3.unique())]

# Keeps countries without official or healthsites.io records
file = file[~file.isoalpha3.isin(off_name + hsite_name)]

# Create variables
file['source']    = "OSM"
file['source_id'] = file.id

# Keep variables of interest
file = file[['isoalpha3','source','source_id','amenity','name','lat','lon']]

# Add to master table
healthcare_inf.append(file)

#### 4.2.4. Master table

In [19]:
# Generate master table 
healthcare = pd.concat(healthcare_inf)
healthcare = healthcare.reset_index(drop = True)

In [20]:
%%capture
# Export to Data Lake 
path = scldatalake + f"{path}/healthcare_facilities.csv"
healthcare.to_csv(path, compression = 'gzip')